In [128]:
import pandas as pd
import numpy as np
import matplotlib
import os

In [210]:
df_train = pd.read_csv("/Users/zamaletdinovaaa/mipt_cet_hack/Silicondzor/data/train.csv", "|")
df_train_sorted = df_train.sort_values(by=['well id', 'depth, m'])
df_train_sorted.to_csv('train_sorted.csv',index=False, sep='|')

In [217]:
df_train_clean = df_train.drop_duplicates(subset=['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)
df_train_clean.to_csv('train_clean.csv',index=False, sep='|')
df_train_clean_sorted = df_train_clean.sort_values(by=['well id', 'depth, m'])
df_train_clean_sorted.to_csv('train_clean_sorted.csv',index=False, sep='|')

In [130]:
df_test = pd.read_csv("/Users/zamaletdinovaaa/mipt_cet_hack/Silicondzor/data/test.csv", "|")

In [169]:
_PNB = 4150
_PHI = 0.7
_RHO = 860
_S = 100
_H_DELTA = 0.1
_EXPENCES_BK = 2450
_EXPENCES_GZ = 2050
_EXPENCES_DGK = 1300
_EXPENCES_NKT = 2050
_EXPENCES_ALPS = 1150
_EXPENCES_DICT = {'bk':_EXPENCES_BK, 
                 'GZ1':_EXPENCES_GZ,
                  'GZ2':_EXPENCES_GZ,
                  'GZ3':_EXPENCES_GZ,
                  'GZ4':_EXPENCES_GZ,
                  'GZ5':_EXPENCES_GZ,
                  'GZ6':_EXPENCES_GZ,
                  'GZ7':_EXPENCES_GZ,
                  'DKG':_EXPENCES_DGK,
                  'NKTD':_EXPENCES_NKT,
                  'NKTM':_EXPENCES_NKT,
                  'NKTR':_EXPENCES_NKT,
                  'ALPS':_EXPENCES_ALPS
                 }
_EXPENCES_COLUMNS_ALL = list(_EXPENCES_DICT.keys())

In [153]:
def analysis_columns(df):
    return list(pd.Index(_EXPENCES_COLUMNS_ALL).intersection(df.columns))
    
def add_expences(df):
    df['expences'] = (df[analysis_columns(df)].fillna(0) * [_EXPENCES_DICT.get(key) for key in analysis_columns(df)]).sum(axis=1)
    return df

In [167]:
def form_result_df(df):
    df_grouped = df.groupby(['well id'])
    columns = ['well id', 'total_cnt', 'oil_cnt', 'value', 'expenses', 'square', 'mass', 'cost']

    df_dashbord_show = pd.DataFrame(columns=columns)
    
    for well_id, df_well in df_grouped:
        total_cnt = df_well.shape[0]
        oil_cnt = df_well[df_well['goal']==1]['well id'].count()
        h = _H_DELTA * oil_cnt
        expences = df_well['expences'].sum()
        square = oil_cnt*_S
        mass = _H_DELTA * square * _RHO
        value = _PNB*_PHI*h*_S*_RHO - expences
        cost = mass * _PNB
        row_dict = [{'cost':cost,'mass':mass,'square':square,'well id':well_id, 'total_cnt':total_cnt, 'oil_cnt':oil_cnt, 'value':value, 'expenses':expences}]
        df_dashbord_show_inc = pd.DataFrame(row_dict)
        df_dashbord_show = df_dashbord_show.append(df_dashbord_show_inc)
    return df_dashbord_show

In [154]:
df_train = add_expences(df_train)    

In [156]:
df_test = add_expences(df_train)    

In [170]:
df_train_result = form_result_df(df_train)

In [181]:
df_train_result.head()

,cost,expenses,mass,oil_cnt,square,total_cnt,value,well id
0,6.281440e+09,6.261313e+06,1513600.0,176,17600,696,4.390747e+09,1.0
0,4.496940e+09,1.342072e+07,1083600.0,126,12600,1472,3.134437e+09,2.0
0,4.068660e+09,1.095490e+07,980400.0,114,11400,1167,2.837107e+09,3.0
0,4.532630e+09,1.066882e+07,1092200.0,127,12700,1170,3.162172e+09,4.0
0,1.070700e+10,1.685752e+07,2580000.0,300,30000,1844,7.478042e+09,5.0


,well id,"depth, m",bk,GZ1,GZ2,GZ3,GZ4,GZ5,GZ7,DGK,NKTD,NKTM,NKTR,ALPS,lith,goal,expences
40248,12.0,3047.833,0.439828,0.284464,0.143336,0.832348,0.168417,0.519864,0.661168,0.366317,0.369366,0.594745,0.393943,0.378775,ARGILLIT,0.0,1077.578281
40249,12.0,3047.833,0.439828,0.284464,0.143336,0.832348,0.168417,0.519864,0.661168,0.366317,0.369366,0.594745,0.393943,0.378775,SAND,1.0,1077.578281
43233,12.0,3043.537,0.175659,0.774000,0.774000,0.774000,0.421732,0.774000,0.984925,0.166929,0.456618,0.591548,0.329737,0.578182,NK,0.0,430.363687
43234,12.0,3043.537,0.175659,0.774000,0.774000,0.774000,0.421732,0.774000,0.984925,0.166929,0.456618,0.591548,0.329737,0.578182,ARGILLIT,0.0,430.363687
46527,12.0,2970.863,0.769854,0.522222,0.263217,0.121967,0.152143,0.479913,0.945325,0.151356,0.492625,0.586432,0.311839,0.427739,DENSE,0.0,1886.141955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172501,12.0,3000.756,0.219189,0.746288,0.328660,0.236672,0.353167,0.144772,0.372647,0.498727,0.284443,0.471454,0.555759,0.155852,ARGILLIT,0.0,537.012206
177488,12.0,3006.663,0.227668,0.152125,0.651959,0.337835,0.468252,0.138662,0.462414,0.491514,0.319664,0.466472,0.442339,0.161782,ARGILLIT,0.0,557.787455
177489,12.0,3006.663,0.227668,0.152125,0.651959,0.337835,0.468252,0.138662,0.462414,0.491514,0.319664,0.466472,0.442339,0.161782,ARGILLIT,0.0,557.787455
204402,12.0,2998.608,0.238214,0.133968,0.551626,0.327296,0.617170,0.187546,0.231582,0.515214,0.276895,0.439736,0.482830,0.139331,ARGILLIT,0.0,583.624908


In [176]:
df_grouped = df_train.groupby(['well id'])

for well_id, df_well in df_grouped:
    print(f'well_id {well_id} nunique - {df_well["depth, m"].nunique()} shape {df_well.shape}')
          
          
df1 = df[df['well id']==12]
df1[df1.duplicated(['depth, m',], keep=False)]

well_id 1.0 nunique - 681 shape (696, 17)
well_id 2.0 nunique - 1444 shape (1472, 17)
well_id 3.0 nunique - 1130 shape (1167, 17)
well_id 4.0 nunique - 1121 shape (1170, 17)
well_id 5.0 nunique - 1772 shape (1844, 17)
well_id 6.0 nunique - 2165 shape (2191, 17)
well_id 7.0 nunique - 528 shape (555, 17)
well_id 8.0 nunique - 1448 shape (1469, 17)
well_id 9.0 nunique - 457 shape (480, 17)
well_id 10.0 nunique - 1414 shape (1431, 17)
well_id 11.0 nunique - 1430 shape (1462, 17)
well_id 12.0 nunique - 1132 shape (1167, 17)
well_id 13.0 nunique - 613 shape (632, 17)
well_id 14.0 nunique - 626 shape (643, 17)
well_id 15.0 nunique - 5081 shape (5114, 17)
well_id 16.0 nunique - 544 shape (553, 17)
well_id 17.0 nunique - 540 shape (550, 17)
well_id 18.0 nunique - 549 shape (568, 17)
well_id 19.0 nunique - 489 shape (505, 17)
well_id 20.0 nunique - 2747 shape (2827, 17)
well_id 21.0 nunique - 486 shape (502, 17)
well_id 22.0 nunique - 535 shape (550, 17)
well_id 23.0 nunique - 1246 shape (1285, 

In [204]:
df2['well id'].isna().count()

21881

In [190]:
df1=df

In [200]:
df2=df1[df1.duplicated(['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)]




In [ ]:
df.drop_duplicates(subset=['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)